In [14]:
import preprocessing_pipeline as pp
import mne
import numpy as np
import pickle

In [15]:
def load_raw_data(path):
    raw = pp.load_eeg_data(path, eeg_format="cnt", use_montage_creator=True, montage_type='standard_1020')
    raw = pp.add_stimulus_to_raw(raw)
    raw = pp.bandpass_filter(raw)
    raw = pp.average_rereference(raw)

    events = mne.find_events(raw, stim_channel='STI')
    return raw, events

In [32]:
def get_need_to_search_event_times(events):
    #Load in the IDs specific to word displays, as well as NeedToSearch conditional IDs
    NeedToSearchIDs = [21,22,23,24,25,26,27,28,29,30,31,32,33,4,5,6,9]

    events_times = mne.pick_events(events,include=NeedToSearchIDs)
    events_times = np.where(events_times == 4, 34, events_times) #Change 4 & 5 to 34 & 35 to allow loop to run
    events_times = np.where(events_times == 5, 35, events_times)
    events_times = np.where(events_times == 6, 36, events_times) #Change 6 & 9 to 34 & 35 to allow loop to run
    events_times = np.where(events_times == 9, 37, events_times)

    #Segment all trials into lists of events
    event_sequence_list = []
    event_sequence = []
    event_id_max = 0
    for x in events_times:
        event_info = x
        event_id = event_info[2]
        #print(event_id_max)
        if event_id > event_id_max:
            event_id_max = event_id
            event_sequence.append(event_info)
        else:
            event_sequence_list.append(event_sequence)
            event_sequence = []
            event_id_max = 0

    #Find all trials that contain both 34 and 35 e.g. NeedToSearch conditions
    NeedToSearchEventTimes = []
    CorrectEventTimes = []
    InCorrectEventTimes = []
    for x in event_sequence_list:
        event_sequence = x
        if any(36 and 37 in array for array in event_sequence) == True:
            NeedToSearchEventTimes.append(event_sequence)
        elif any(34 in array for array in event_sequence) == True:
            CorrectEventTimes.append(event_sequence)
        elif any(35 in array for array in event_sequence) == True:
            InCorrectEventTimes.append(event_sequence)

    return NeedToSearchEventTimes, CorrectEventTimes, InCorrectEventTimes



In [66]:
def get_eeg_time_segments(raw, EventTimes):
    raw_copy = raw.copy()
    sample_rate = 500 #TODO - get this from raw
    eeg_time_segments = []
    egg_event_ids = []
    for event_sequence in EventTimes:
        for event in event_sequence:
            start_index = event[0]

            end_index = int(start_index + (0.8*sample_rate))
            cropped_data = raw_copy[:, start_index:end_index]
            eeg_time_segments.append(cropped_data)
            egg_event_ids.append(event[2])
    return eeg_time_segments, egg_event_ids

In [18]:
participant_number = "01"
path = fr"I:\Science\CIS-YASHMOSH\niallmcguire\dominika\Raw Data\0{participant_number}\0{participant_number}.cnt"
raw, events = load_raw_data(path)


Reading 0 ... 3507399  =      0.000 ...  7014.798 secs...


C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\preprocessing_pipeline.py:137: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = mne.io.read_raw_cnt(file, preload=preload)
C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\preprocessing_pipeline.py:53: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['A1', 'A2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  self.raw.set_montage(montage)


Creating RawArray with float64 data, n_channels=1, n_times=3507400
    Range : 0 ... 3507399 =      0.000 ...  7014.798 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.25 Hz)
- Filter length: 16501 samples (33.002 sec)



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:   10.1s
[Parallel(n_jobs=4)]: Done  34 out of  34 | elapsed:   16.8s finished


Applying average reference.
Applying a custom ('EEG',) reference.
1839 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  20  21  22  23
  24  25  26  27  28  29  30  31  32  33  34  35  36  39  44  55  56  57
  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129
 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
 148 149 150 151 152 154 155 156 157 158 159 160 161 162 163 164 165 166
 167 168 169 170 171 173 174 175]


In [64]:
NeedToSearchEventTimes, CorrectEventTimes, InCorrectEventTimes  = get_need_to_search_event_times(events)

In [68]:
#NeedToSearchEEGSegments, NeedToSearchEventIDs = get_eeg_time_segments(raw, NeedToSearchEventTimes)
#CorrectEventTimes, CorrectEventIDs = get_eeg_time_segments(raw, CorrectEventTimes)
InCorrectEventTimes, InCorrectEventIDs = get_eeg_time_segments(raw, InCorrectEventTimes)

TypeError: only length-1 arrays can be converted to Python scalars

In [53]:
InCorrectEventTimes

[[array([85499,     0,    22], dtype=int64),
  array([86299,     0,    23], dtype=int64),
  array([87099,     0,    24], dtype=int64),
  array([87899,     0,    25], dtype=int64),
  array([88699,     0,    26], dtype=int64),
  array([89499,     0,    27], dtype=int64),
  array([90299,     0,    28], dtype=int64),
  array([91099,     0,    29], dtype=int64),
  array([102477,      0,     35], dtype=int64)],
 [array([110215,      0,     22], dtype=int64),
  array([111015,      0,     23], dtype=int64),
  array([111815,      0,     24], dtype=int64),
  array([129475,      0,     35], dtype=int64)],
 [array([138181,      0,     22], dtype=int64),
  array([138981,      0,     23], dtype=int64),
  array([139781,      0,     24], dtype=int64),
  array([140581,      0,     25], dtype=int64),
  array([141381,      0,     26], dtype=int64),
  array([142181,      0,     27], dtype=int64),
  array([142981,      0,     28], dtype=int64),
  array([143781,      0,     29], dtype=int64),
  array([14458

In [177]:
ParticipantList = ["01", "02", "03", "04", "05", "06", "08", "09", "10",
                   "11", "12", "13", "14", "15", "16", "18", "19", "20",
                   "21", "22", "23", "24"]

for participant_number in ParticipantList:
    path = fr"I:\Science\CIS-YASHMOSH\niallmcguire\dominika\Raw Data\0{participant_number}\0{participant_number}.cnt"
    raw, events = load_raw_data(path)
    NeedToSearchEventTimes = get_need_to_search_event_times(events)
    NeedToSearchEEGSegments = get_eeg_time_segments(raw, NeedToSearchEventTimes)
    pickle_file_path = fr'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\EEG-NeedToSearch-DataSegments\participant_0{participant_number}_EEG_data.pkl'

    # Open the file in binary write mode and use pickle.dump to save the data
    with open(pickle_file_path, 'wb') as pickle_file:
        pickle.dump(NeedToSearchEEGSegments, pickle_file)


Reading 0 ... 3507399  =      0.000 ...  7014.798 secs...


C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\preprocessing_pipeline.py:137: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = mne.io.read_raw_cnt(file, preload=preload)
C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Information-Need\Data\preprocessing_pipeline.py:53: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['A1', 'A2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  self.raw.set_montage(montage)


Creating RawArray with float64 data, n_channels=1, n_times=3507400
    Range : 0 ... 3507399 =      0.000 ...  7014.798 secs
Ready.
1839 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  20  21  22  23
  24  25  26  27  28  29  30  31  32  33  34  35  36  39  44  55  56  57
  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129
 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
 148 149 150 151 152 154 155 156 157 158 159 160 161 162 163 164 165 166
 167 168 169 170 171 173 174 175]


In [188]:
'''
# Specify the file path where you saved the pickle file
pickle_file_path = 'your_file_path.pkl'

# Open the file in binary read mode and use pickle.load to load the data
with open(pickle_file_path, 'rb') as pickle_file:
    loaded_data = pickle.load(pickle_file)

# Now 'loaded_data' contains the data structure you saved
'''